# US Patent

In [1]:
datapath = 'D:/data/patent/ipa200604.xml'

In [2]:
cd ..

C:\Users\simon\OneDrive\Desktop\git_repo\patentAI


In [3]:
from src.datapipeline.preprocess_xml import pipeline

In [4]:
extracted_data = pipeline(datapath)

100%|█████████████████████████████████████████████████████████████████████████████| 8515/8515 [00:43<00:00, 197.95it/s]


In [11]:
extracted_data[8000]

[['1. A method for indicating media viewership, the method comprising:identifying a media content item based at least in part on a viewing history of a first user associated with a user device;receiving a request to present a page that includes a representation of the media content item;determining, for the media content item, a viewership status of users in a group of users connected to the first user, wherein the viewership status corresponds to one of a different level of engagement with the media content item;causing the requested page to be presented on the user device, wherein the requested page includes a first portion that includes the representation of the media content item and a second portion that indicates viewership information corresponding to the media content item, wherein the second portion includes a plurality of groups of indicators each corresponding to a different level of engagement with the media content item, wherein each indicator in the group of indicators re

In [10]:
len(extracted_data)

8515

In [13]:
import requests

# Singapore patent

In [136]:
import datetime
from tqdm import tqdm

In [146]:
numdays = 3650
base = datetime.datetime.today()
date_list = [base - datetime.timedelta(days=x) for x in range(numdays)]
weekday = [date for date in date_list if date.weekday() < 5]
ipos_format = [date.strftime('%Y-%m-%d') for date in weekday]

In [147]:
ipos_api = 'https://api.data.gov.sg/v1/technology/ipos/patents?lodgement_date='

In [148]:
target_doc = []
for date in tqdm(ipos_format):
    api = ipos_api + date
    result = requests.get(api).json()
    applications = result['items']
    for app in applications:
        documents = app['documents']
        for doc in documents:
            if doc['docType']['description'] == 'Description (with claims)':
                target_doc.append(doc['url'])

100%|████████████████████████████████████████████████████████████████████████████| 2608/2608 [1:22:55<00:00,  1.91s/it]


In [149]:
import pickle

target_doc_savepath = 'pkl_files/target_doc_url.pkl'
with open(target_doc_savepath, 'wb') as file:
        pickle.dump(target_doc, file)

In [150]:
target_doc

['https://ipos-storage.data.gov.sg/patents/10202004071U/cff2155d-507d-452a-9585-1d2087ec304b/e1617416-35b4-42d1-a49a-b50f36c971bb.pdf',
 'https://ipos-storage.data.gov.sg/patents/10202004083U/c52678d6-2ad0-4c73-b83b-12c0c006e437/de9d809b-f149-4616-b7fb-ef7edb17b630.pdf',
 'https://ipos-storage.data.gov.sg/patents/10202004091T/ef05c697-904f-4f86-acc5-c524ec919ae8/b9d80a78-6ca0-4d3c-bf3a-9b7100ca946b.pdf',
 'https://ipos-storage.data.gov.sg/patents/10202004094V/48d2187e-5078-4103-affc-4ebce42ed9b4/9f520f97-b487-475a-ba00-2b2b9ad3b6c1.pdf',
 'https://ipos-storage.data.gov.sg/patents/10202004042Q/ca6da1b8-8c6d-4411-98ed-4784a6b9956c/3a95a429-42b6-4e36-9931-aaf74d9db7b4.pdf',
 'https://ipos-storage.data.gov.sg/patents/10202004043W/bc78fa5c-7882-44d6-be40-e4f4b87996ec/89f55457-2069-4529-9989-7b0f25451bd0.pdf',
 'https://ipos-storage.data.gov.sg/patents/10202004044U/af88a558-25ce-478b-a9f9-c06558f5a8f9/ab3f207e-4c21-404f-98f4-a77fec954dff.pdf',
 'https://ipos-storage.data.gov.sg/patents/10202

In [151]:
import requests, PyPDF2, io

def load_web_pdf(url):
    response = requests.get(url)

    with io.BytesIO(response.content) as open_pdf_file:
        read_pdf = PyPDF2.PdfFileReader(open_pdf_file)
        num_pages = read_pdf.getNumPages()
        txt = [read_pdf.getPage(i).extractText() for i in range(num_pages)]
        return txt
    
def extract_intro(txt):
    intro = txt[0:2]
    intro = ' '.join(intro)
    return intro
    
def extract_claim_text(txt):
    claims_start_page = [i for i, page in enumerate(txt) if ('CLAIMS' in page) or ('what is claimed is' in page.lower())][0]
    claim_pages = txt[claims_start_page:]
    text_pages = txt[0:claims_start_page]
    return claim_pages, text_pages

In [154]:
access_problem = []
data = []
failed_extract_text = []

for url in tqdm(target_doc):
    # Load PDF
    try:
        txt = load_web_pdf(url)
    except Exception:
        access_problem.append(url)
        continue
    
    # Extract text
    try:
        intro = extract_intro(url)
        claim_pages, text_pages = extract_claim_text(txt)
        data.append([intro, claim_pages, text_pages, txt])
    except Exception:
        failed_extract_text.append(txt)


combined_saved = [data, failed_extract_text, access_problem]
target_doc_savepath = 'pkl_files/combined_extract_from_pdf.pkl'
with open(target_doc_savepath, 'wb') as file:
        pickle.dump(combined_saved, file)

  0%|                                                                           | 103/86122 [01:08<15:49:17,  1.51it/s]


KeyboardInterrupt: 

In [1]:
pwd

'C:\\Users\\simon\\OneDrive\\Desktop\\git_repo\\patentAI\\notebooks'

In [114]:
ipos_api = 'https://api.data.gov.sg/v1/technology/ipos/patents?lodgement_date=2010-01-22'

In [115]:
result = requests.get(ipos_api).json()

In [116]:
result.keys()

dict_keys(['lodgement_date', 'count', 'items'])

In [119]:
result['items'][1]

{'securityInterest': None,
 'summary': {'applicationNum': '2010004703',
  'applicationType': 'PCT-NP',
  'applicationStatus': 'Abandoned ',
  'publicationPatentNumForOldApplication': '158582 ',
  'titleOfInvention': 'SECONDARY SYNCHRONIZATION CODEBOOK FOR E-UTRAN',
  'filingDate': '2008-08-12',
  'lodgementDate': '2010-01-22',
  'ipc': None,
  'dateOfPublication': None},
 'divisionalApplications': None,
 'declarationOfPriority': [{'applicationNum': '12/166,044',
   'country': {'description': 'United States of America', 'code': 'US'},
   'filingDate': '2008-07-01'},
  {'applicationNum': '12/166,495',
   'country': {'description': 'United States of America', 'code': 'US'},
   'filingDate': '2008-07-02'},
  {'applicationNum': '60/955,623',
   'country': {'description': 'United States of America', 'code': 'US'},
   'filingDate': '2007-08-13'}],
 'pctApplication': [{'pctPublicationNum': 'WO 2009/023670',
   'pctApplicationNu,': 'PCT/US2008/072904',
   'dateOfPublicationOfEntryIntoNationalPh

In [53]:
result['items'][0].keys()

dict_keys(['securityInterest', 'summary', 'divisionalApplications', 'declarationOfPriority', 'pctApplication', 'inventors', 'grantAndRenewal', 'rupka', 'documents', 'license', 'agentCorrespondenceDetails', 'applicationNum', 'applicant', 'hmgStatus', 'transferOfOwnership'])

In [108]:
result['items'][1]['documents'][0]

{'fileName': '2010004703_CSD7838171.pdf',
 'lodgementDate': '2010-02-18',
 'docType': {'description': 'Description (with claims)', 'code': 'DWC'},
 'fileId': '105C4FA1-1759-4485-BCE1-13F820B9FF8F',
 'url': 'https://ipos-storage.data.gov.sg/patents/2010004703/f78a6762-5d86-4a90-8991-1cb2a429d95a/2010004703_CSD7838171.pdf'}

In [111]:
for doc in result['items'][1]['documents']:
    if doc['docType']['description'] == 'Description (with claims)':
        target_doc = doc['url']

In [112]:
target_doc

'https://ipos-storage.data.gov.sg/patents/2010004703/f78a6762-5d86-4a90-8991-1cb2a429d95a/2010004703_CSD7838171.pdf'

In [29]:
result = requests.get('https://ipos-storage.data.gov.sg/patents/2010004711/053507fb-55ba-41de-8029-ecfc3cdd66b4/2010004711_CSD7838044.pdf')

In [56]:
url = 'https://ipos-storage.data.gov.sg/patents/2010004703/f78a6762-5d86-4a90-8991-1cb2a429d95a/2010004703_CSD7838171.pdf'

In [57]:
import requests, PyPDF2, io

response = requests.get(url)

with io.BytesIO(response.content) as open_pdf_file:
    read_pdf = PyPDF2.PdfFileReader(open_pdf_file)
    num_pages = read_pdf.getNumPages()
    txt = [read_pdf.getPage(i).extractText() for i in range(num_pages)]

In [71]:
claims = [[i, page] for i, page in enumerate(txt) if 'CLAIMS' in page]

In [72]:
claims = claims[0]
claims

[38,
 "\n[00125] System 1400 can comprise a module 1402 for\nforming a sequence matrix from a base M-sequence and n\ncyclic shifted variations of the base M-sequence, as\ndescribed herein. Further, system 1400 can comprise a\nmodule 1404 for indexing sequence pairs of the\nsequence matrix. The module can generate at least\n(n+l) L2 indices for each distinct sequence pair of\nthe sequence matrix. Furthermore, system 1400 can\ncomprise a module 1406 for determining PAPR and/or\ncorrelation of SSC codes resulting from sequence\npairs. The module 1406 can select a predetermined\nnumber of sequence pairs {e.g., substantially 170\nsequence pairs, substantially 340 sequence pairs, or\nother suitable number based at least in part on a\nnumber of basestations in a mobile site etc.) meeting\nPAPR and/or cross-correlation thresholds {e.g., below\na desired PAPR and/or below a desired correlation\nfactor). Accordingly, SSCs resulting from the selected\nsequence pairs can have desired transmission\

In [73]:
claims_pattern = r'CLAIMS.+'

In [74]:
claim_text = ' '.join(txt[claims:])

TypeError: slice indices must be integers or None or have an __index__ method

In [79]:
claim_text = claim_text.replace('\n', ' ').strip()

In [80]:
import re

In [81]:
raw_claims = re.findall(claims_pattern, claim_text)

In [86]:
raw_des = re.findall(r'.+CLAIMS', ' '.join(txt).replace('\n', ' ').strip())

In [88]:
raw_des

['SECONDARY SYNCHRONIZATION CODEBOOK FOR E-UTRAN CROSS REFERENCE TO RELATED APPLICATION [0001] The present Non-provisional Application for Patent claims priority to Provisional Patent Application No. 60/955,623 filed on August 13, 2007 and entitled SECONDARY SYNCHRONIZATION CODEBOOK FOR E-UTRAN, assigned to the assignee hereof and hereby expressly incorporated by reference herein. BACKGROUND I. Field [0002] The following relates generally to wireless communication, and more specifically to determining a secondary synchronization codebook for selecting secondary synchronization codes for a radio network site. II. Background [0003] Wireless communication systems are widely deployed to provide various types of communication content such as, e.g., voice content, data content, and so on. Typical wireless communication systems can be multiple-access systems capable of supporting communication with multiple users by sharing available system resources (e.g., bandwidth, transmit power). Example

In [89]:
import datetime

In [99]:
numdays=365
base = datetime.datetime.today()
date_list = [base - datetime.timedelta(days=x) for x in range(numdays)]

In [100]:
date_list

[datetime.datetime(2020, 6, 4, 17, 29, 2, 246474),
 datetime.datetime(2020, 6, 3, 17, 29, 2, 246474),
 datetime.datetime(2020, 6, 2, 17, 29, 2, 246474),
 datetime.datetime(2020, 6, 1, 17, 29, 2, 246474),
 datetime.datetime(2020, 5, 31, 17, 29, 2, 246474),
 datetime.datetime(2020, 5, 30, 17, 29, 2, 246474),
 datetime.datetime(2020, 5, 29, 17, 29, 2, 246474),
 datetime.datetime(2020, 5, 28, 17, 29, 2, 246474),
 datetime.datetime(2020, 5, 27, 17, 29, 2, 246474),
 datetime.datetime(2020, 5, 26, 17, 29, 2, 246474),
 datetime.datetime(2020, 5, 25, 17, 29, 2, 246474),
 datetime.datetime(2020, 5, 24, 17, 29, 2, 246474),
 datetime.datetime(2020, 5, 23, 17, 29, 2, 246474),
 datetime.datetime(2020, 5, 22, 17, 29, 2, 246474),
 datetime.datetime(2020, 5, 21, 17, 29, 2, 246474),
 datetime.datetime(2020, 5, 20, 17, 29, 2, 246474),
 datetime.datetime(2020, 5, 19, 17, 29, 2, 246474),
 datetime.datetime(2020, 5, 18, 17, 29, 2, 246474),
 datetime.datetime(2020, 5, 17, 17, 29, 2, 246474),
 datetime.dateti

In [101]:
weekday = [date for date in date_list if date.weekday() < 5]
weekday

[datetime.datetime(2020, 6, 4, 17, 29, 2, 246474),
 datetime.datetime(2020, 6, 3, 17, 29, 2, 246474),
 datetime.datetime(2020, 6, 2, 17, 29, 2, 246474),
 datetime.datetime(2020, 6, 1, 17, 29, 2, 246474),
 datetime.datetime(2020, 5, 29, 17, 29, 2, 246474),
 datetime.datetime(2020, 5, 28, 17, 29, 2, 246474),
 datetime.datetime(2020, 5, 27, 17, 29, 2, 246474),
 datetime.datetime(2020, 5, 26, 17, 29, 2, 246474),
 datetime.datetime(2020, 5, 25, 17, 29, 2, 246474),
 datetime.datetime(2020, 5, 22, 17, 29, 2, 246474),
 datetime.datetime(2020, 5, 21, 17, 29, 2, 246474),
 datetime.datetime(2020, 5, 20, 17, 29, 2, 246474),
 datetime.datetime(2020, 5, 19, 17, 29, 2, 246474),
 datetime.datetime(2020, 5, 18, 17, 29, 2, 246474),
 datetime.datetime(2020, 5, 15, 17, 29, 2, 246474),
 datetime.datetime(2020, 5, 14, 17, 29, 2, 246474),
 datetime.datetime(2020, 5, 13, 17, 29, 2, 246474),
 datetime.datetime(2020, 5, 12, 17, 29, 2, 246474),
 datetime.datetime(2020, 5, 11, 17, 29, 2, 246474),
 datetime.dateti

In [102]:
ipos_format = [date.strftime('%Y-%M-%d') for date in weekday]

In [103]:
ipos_format

['2020-29-04',
 '2020-29-03',
 '2020-29-02',
 '2020-29-01',
 '2020-29-29',
 '2020-29-28',
 '2020-29-27',
 '2020-29-26',
 '2020-29-25',
 '2020-29-22',
 '2020-29-21',
 '2020-29-20',
 '2020-29-19',
 '2020-29-18',
 '2020-29-15',
 '2020-29-14',
 '2020-29-13',
 '2020-29-12',
 '2020-29-11',
 '2020-29-08',
 '2020-29-07',
 '2020-29-06',
 '2020-29-05',
 '2020-29-04',
 '2020-29-01',
 '2020-29-30',
 '2020-29-29',
 '2020-29-28',
 '2020-29-27',
 '2020-29-24',
 '2020-29-23',
 '2020-29-22',
 '2020-29-21',
 '2020-29-20',
 '2020-29-17',
 '2020-29-16',
 '2020-29-15',
 '2020-29-14',
 '2020-29-13',
 '2020-29-10',
 '2020-29-09',
 '2020-29-08',
 '2020-29-07',
 '2020-29-06',
 '2020-29-03',
 '2020-29-02',
 '2020-29-01',
 '2020-29-31',
 '2020-29-30',
 '2020-29-27',
 '2020-29-26',
 '2020-29-25',
 '2020-29-24',
 '2020-29-23',
 '2020-29-20',
 '2020-29-19',
 '2020-29-18',
 '2020-29-17',
 '2020-29-16',
 '2020-29-13',
 '2020-29-12',
 '2020-29-11',
 '2020-29-10',
 '2020-29-09',
 '2020-29-06',
 '2020-29-05',
 '2020-29-